## Sequence Extraction from HoloBee Database v2016.1

Import python modules

In [ ]:
import os
import re
import Bio.SeqIO as sio
import pandas as pd

In the function *os.listdir()*, add the file path to the folder containing the **.fasta** files *(HB_Mop_v2016.1.fasta, HB_Bar_v2016.1.fasta)*

In [ ]:
input_filenames = os.listdir(path = 'C:/Users/vishwakarmas/Downloads/HB_v2016.1/HB_v2016.1')
input_name = [input_file for input_file in input_filenames if re.match(r"(^.*.fasta)$", input_file, flags = re.IGNORECASE)]
print(input_name)

A *for* loop is used to perform iterations to the files that end with **.fasta** and pull Descriptions from *sequence id* using a parse function from *Bio.SeqIO.parse()*

In [ ]:
def split_seq(sequence):
    return sequence.split("|")

def split_comma(sequence):
    return sequence[(len(sequence) - 1)].split(",")

In [ ]:
for input_file in input_name:
    seq_list_obj = sio.parse(input_file, "fasta")
    seq_de = [item.description for item in seq_list_obj]
    
    seq_de_split = [split_seq(seq) for seq in seq_de]
    info_type = [split_comma(seq) for seq in seq_de_split]
    seq_info = [seq[0].strip() for seq in info_type]
    
    seq_seq = [item[1].strip() if len(item) > 1 else "-" if len(item) == 1 else "Description Unknown" for item in info_type]

    seq_id = [seq.split(" ")[0] for seq in seq_de]
    
    df = pd.DataFrame(data = {"Sequence_ID": seq_id, "Description": seq_info, "Sequencing": seq_seq})
        
    output_name = str(input_file.split(".")[0]) + "_Sequence_Info_Jupyter.csv"
    print(output_name)
        
    df.to_csv(output_name, sep = ',', index = False)

    seq_list_obj.close()

### View preliminary information
Using the output file created, the first 2 words from the Description column is used to create a bar plot

Import python modules and view files present in the file path for the output file *(_Sequence_Info_Jup.csv)*

In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

file_path = os.listdir(path = 'C:/Users/vishwakarmas/Downloads/HB_v2016.1/HB_v2016.1')

From the list of the files present in the file path, filter the files with the following pattern: *_Sequence_Info_Jup.csv*

In [ ]:
input_name_graph = [input_file for input_file in file_path if re.match(r"(^.*_Sequence_Info_Jup.csv)$", input_file, flags = re.IGNORECASE)]
print(input_name_graph)

Enter a file from the list above in the **pd.read_csv("*file_name.csv*")** function

In [ ]:
vis_info = pd.read_csv("HB_Mop_v2016_Sequence_Info_Jup.csv")

Pull the data from the *vis_info* column *Description* and create a dataframe and name the column to *Full_des*

In [ ]:
des_re = pd.DataFrame(vis_info['Description'].unique()).rename(columns = {0 : "Full_des"})
print(des_re)

Split the values in the description into 3 elements by empty spaces

In [ ]:
t2 = des_re['Full_des'].str.split(" ", n = 2)
print(t2)

Use a for loop to concatenate the first 2 elements and append it to the empty list

In [ ]:
t2_loc = t2.to_list()

list_element = []
item_list = []
for element in t2_loc:
    list_element.append(element)
    el_1 = element[0]
    el_2 = element[1]
    item = el_1 + " " + el_2
    item_list.append(item)

View the list of the first 2 words of the description

In [ ]:
#print(item_list)

Create a new column with the data from the "item_list" list to the "des_re" dataframe, creating a dataframe with 2 columns.

In [ ]:
df = pd.concat([des_re, pd.Series(item_list, name = "Short_des")], axis = 1)

A brief overview of the first 2 words from the description present in the sequence

In [ ]:
df2 = pd.Series(item_list, name = "Short_des").value_counts()
print(df2)

View bar plot for the data created from the first 2 words from the description present in the sequencing file

In [ ]:
plt.bar(df2.index.values, height = df2, width = 0.9, color = "red")